# get ask coposite or not response from gemini

In [1]:
# !pip install -q -U google-generativeai
# !pip install python-dotenv

In [2]:
import os
from dotenv import load_dotenv
import random
import time
import json
import google.generativeai as genai

### load model

In [3]:
load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [4]:
gemini = genai.GenerativeModel('gemini-pro')

### load data and create initial prompt

In [5]:
with open('../../data/primality_test.json','r') as file:
    data = json.load(file)
primes = data['primes']
composites = data['composites']

# mix primes and composites so model is not biased
data = primes + composites
random.seed(42)
random.shuffle(data)

# create batches, batch sizes are heuristics
# batch sizes, although input token limit is 30,720 tokens, output token limit is 2048
# which restricts the batch size

batch_prompts = []
BATCH_SIZE = 100
index = 0

while index < len(data):
    batch = [data[i] for i in range(index, min(index+BATCH_SIZE, len(data)))]
    batch_prompts.append(batch)
    index += BATCH_SIZE
    
print("Number of batches:", len(batch_prompts))
print("Number of prompts in each batch:", [len(batch) for batch in batch_prompts])
        

Number of batches: 100
Number of prompts in each batch: [100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100]


In [12]:
# give a prompt once and then generate the response,
# and then use this prompt+response directly instead of doing everytime

promt_saying_you_are_a_math_assistant = '''You are a math assistant. I will ask you some questions. Please answer in the correct format. For example, if I ask 'Is 5 a composite number?' , you should answer '5:Yes' if 5 is a composite number or '5:No' if 5 is not a composite number. Each question is in a separate line. Please return each answer in a separate line. Do you understand ?'''
Initial_messages = [{'role':'user', 'parts': [promt_saying_you_are_a_math_assistant] }]

# get response from the model and then append it to the prompt
Initial_response = gemini.generate_content(Initial_messages)                                   
Initial_messages.append(Initial_response.candidates[0].content)

In [13]:
print(Initial_messages)

[{'role': 'user', 'parts': ["You are a math assistant. I will ask you some questions. Please answer in the correct format. For example, if I ask 'Is 5 a composite number?' , you should answer '5:Yes' if 5 is a composite number or '5:No' if 5 is not a composite number. Each question is in a separate line. Please return each answer in a separate line. Do you understand ?"]}, parts {
  text: "Yes, I understand. I will answer each question in the format \'n:Yes\' if n is a composite number or \'n:No\' if n is not a composite number, where n is the number in the question."
}
role: "model"
]


In [14]:
import pickle

# save the initial messages to a file, in case we need to restart the conversation, initial message should be consistent
# across all batches

with open('initial_messages_ask_composite.pkl', 'wb') as file:
    pickle.dump(Initial_messages, file)

### conduct experiment 

In [15]:
# path to save responses
response_file = 'gemini_response/ask_composite_response.txt'

# request per minute limit for free api
rpm = 60

In [16]:
# to start from a specific batch number, give printed_batch_num -1 here
batch_no_to_start_from = 0

for i, batch_prompt in enumerate(batch_prompts):

    if i < batch_no_to_start_from:
        continue

    # print status
    print(f"Sending batch {i+1} of {len(batch_prompts)}")

    num_prompts = len(batch_prompt)

    # create prompt for the batch
    curr_prompt = "".join([f"Is {prompt} a composite number?\n" for prompt in batch_prompt])

    
    # create a complete prompt to send to the model
    complete_promt = Initial_messages.copy()
    complete_promt.append({'role':'user', 'parts': [curr_prompt]})

    # get response from the model, candidate_count is 1 because we only need one response,
    # temperature is 0 because we need deterministic responses
    response = gemini.generate_content( complete_promt,
                                        generation_config= genai.GenerationConfig(
                                        candidate_count = 1,
                                        temperature = 0.0,
                                        )
                                )
    
    try:
        # extract text from the response
        model_responses = response.candidates[0].content.parts.__getitem__(0).text.split('\n')
    except IndexError:
        time.sleep(5)
        # get response from the model, candidate_count is 1 because we only need one response,
        # temperature is 0 because we need deterministic responses
        response = gemini.generate_content( complete_promt,
                                            generation_config= genai.GenerationConfig(
                                            candidate_count = 1,
                                            temperature = 0.0,
                                            )
                                    )
        
        try:
            # extract text from the response
            model_responses = response.candidates[0].content.parts.__getitem__(0).text.split('\n')
        except IndexError:
            # if model gives no response, fill with 0
            print("Error in response")
            model_responses = ["0"]*num_prompts

    # save responses to file
    with open(response_file, 'a') as file:
        for model_response in model_responses:
            file.write(model_response + '\n')

    
    #print the prompt and response separated by space
    print('# of prompts: ',len(batch_prompt),'# of responses: ', len(model_responses))


    for prompt, response in zip(batch_prompt, model_responses):
        print(prompt, response, sep=" ->")
    
    
    print("Received responses for batch", i+1, "of", len(batch_prompts))
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

    time.sleep(60//rpm) #to avoid rate limit errors

print("Responses saved to", response_file)

Sending batch 1 of 100
# of prompts:  100 # of responses:  100
54631 ->54631:Yes
22092 ->22092:Yes
99194 ->99194:Yes
85213 ->85213:Yes
81041 ->81041:Yes
81371 ->81371:Yes
2807 ->2807:Yes
91499 ->91499:Yes
96079 ->96079:Yes
17593 ->17593:Yes
70181 ->70181:Yes
47521 ->47521:Yes
86717 ->86717:Yes
89882 ->89882:Yes
34403 ->34403:Yes
61530 ->61530:Yes
40093 ->40093:Yes
3547 ->3547:Yes
89293 ->89293:Yes
46349 ->46349:Yes
31147 ->31147:Yes
61782 ->61782:Yes
72469 ->72469:Yes
71153 ->71153:Yes
5900 ->5900:Yes
91616 ->91616:Yes
74131 ->74131:Yes
98547 ->98547:Yes
58427 ->58427:Yes
43872 ->43872:Yes
35449 ->35449:Yes
41188 ->41188:Yes
25997 ->25997:Yes
95603 ->95603:Yes
14434 ->14434:Yes
32063 ->32063:Yes
90264 ->90264:Yes
373 ->373:Yes
80933 ->80933:Yes
14321 ->14321:Yes
24379 ->24379:Yes
93557 ->93557:Yes
10687 ->10687:Yes
43983 ->43983:Yes
62718 ->62718:Yes
62141 ->62141:Yes
41687 ->41687:Yes
17123 ->17123:Yes
62347 ->62347:Yes
81701 ->81701:Yes
84451 ->84451:Yes
19762 ->19762:Yes
42437 ->424